Title: Predicting tennis player rank categories based on the number of seasons active, prize money won, age, and height.

Background information:
Tennis rankings are determined based on the number of points a player has. Points are acquired by playing in tournaments. Winning higher-status tournaments, such as a Grand Slam, results in more points gained (2000). Winning tournaments also lets players earn prize money, where like points, winning a higher-status tournament gives more prize money. Higher-ranking players are therefore more likely to have earned more prize money. Additionally, players who have competed in more seasons have had more opportunities to win tournaments, possibly indicating that they have a higher ranking



Question: 
The question we will be exploring in this project is: Can we use known player statistics (number of seasons active, prize money, age, and height) of the top 500 players to predict which category rank a future player will reside i? 

To answer this question we will be using the “Player Stats for the Top 500 Players” data frame whi ch provides statistics on each top player such as age, height, number of seasons active, and current ra


Variables:

We will be using the number of seasons a player has played in, how much prize money they've won, the age of the player, and the height of the player.


ayer
nk.
.




In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readxl)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [7]:
tennis_data <- read_csv("Data/player_stats.csv", col_names = FALSE)

tennis_select <- tennis_data |>
    select(X2, X6, X10, X11, X14)
tennis_select

Rows: 501 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (37): X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16,...
dbl  (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X2,X6,X10,X11,X14
<chr>,<chr>,<chr>,<chr>,<chr>
Age,Current Rank,Prize Money,Height,Seasons
26 (25-04-1993),378 (97),NA,NA,NA
18 (22-12-2001),326 (119),"$59,040",NA,NA
32 (03-11-1987),178 (280),"US$3,261,567",185 cm,14
21 (29-05-1998),236 (199),"$374,093",NA,2
27 (21-10-1992),183 (273),"US$6,091,971",193 cm,11
22 (11-02-1997),31 (1398),"$1,517,157",NA,5
28 (18-11-1991),307 (131),"$278,709",NA,1
21 (12-05-1998),232 (205),"$59,123",NA,1
